# Qiskit_qcware basic demo

This is a simple sample notebook showing the basic functionality of the Qiskit_qcware provider.

In order to show that the provider can function across a variety of circuits, we use property-based
testing to generate a variety of circuits, with different numbers of qubits, different numbers of 
classical bits, and various instructions.

Since the translation library programmatically scans Qiskit for available gates, the initial import generates some deprecation warnings which will go away as Qiskit is updated.

In [ ]:
from tests.strategies.qiskit import circuits, gates
from qcware_transpile.dialects.qiskit.qiskit_dialect import dialect

# Generation of Circuits

The following cell uses a strategy to generate random circuits; these circuits are fairly small, but cover a variety of cases.

Run the cell several times to see a sampling of the circuits used for testing!

In [ ]:
# We don't handle reset gates at all, so we'll remove that possibility from the set of generated circuits
allowed_gates = sorted(dialect().gate_defs - {dialect().gate_named('reset')})
qiskit_circuit=circuits(min_qubits=2, max_qubits=4, min_length=2, max_length=4, gates=gates(gate_list=allowed_gates)).example()
qiskit_circuit.draw()

# Testing Statevector Compatibility

To show that the statevectors generated by Aer and Qiskit_qcware are comparable, we'll use a `local_statevector` backend for qiskit_qcware and the basic aer `statevector_simulator` and compare _probability vectors_ within a tolerance.

The reason we compare probability vectors (the absolute value of the statevector) is that some transpilation steps in qiskit seem to introduce global phase changes that are difficult to reconcile.  However, since global phase changes do not affect the probability of a state being read, this should not affect the outcome.

What _can_ affect the outcome is a mid-circuit measurement gate.  Quasar (QCWare's simulator) is designed to produce a statevector at the end of computation and sample from that statevector; aer takes measurements into account and produces statevectors congruent with previous measurements.

To determine if statevectors are "identical", we compare each state's probability; they should be identical within a tolerance.  We arbitrarily choose this tolerance to be 1e-5.

In [ ]:
from qiskit_qcware import QcwareProvider
import qiskit
import numpy

In [ ]:
def qcware_local_probability_vector(circuit: qiskit.QuantumCircuit):
    backend = QcwareProvider().get_backend('local_statevector')
    sv = qiskit.execute(circuit, backend).result().data()['statevector']
    return abs(sv)


def aer_probability_vector(circuit: qiskit.QuantumCircuit):
    backend = qiskit.Aer.get_backend('statevector_simulator')
    sv = qiskit.execute(circuit, backend).result().data()['statevector']
    return abs(sv)


In [ ]:
qiskit_circuit=circuits(min_qubits=2, max_qubits=4, min_length=2, max_length=4, gates=gates(gate_list=allowed_gates)).example()
print(qiskit_circuit.draw())
qcware_pv = qcware_local_probability_vector(qiskit_circuit)
aer_pv = aer_probability_vector(qiskit_circuit)
atol=1e-5
print(f"PVs identical within {atol}: {numpy.allclose(qcware_pv, aer_pv, atol=atol)}")

# Using the Forge backend

We should get similar results using the hosted Forge backend.  The `forge_statevector` backend uses QCWare's gpu-accelerated quantum simulator backend.

For the small circuits in this demo, the added overhead of translation, DNS lookup, and an over-the-wire call makes this feel slow, but the simulator is actually very fast, so for large circuits there is a noticeable improvement.  There is ongoing work to optimize the translation step as well.

## Minor difference between local and remote

The remote GPU-accelerated Forge backend cannot natively handle gates with more than 2 qubits.  As a result, the CSWAP and CCX gates which are in the list of basis gates served by the local simulator are removed from the list of basis gates for the Forge simulator.  This is a reasonable compromise for the increased execution speed at high qubit counts.

## Configuration

Currently the QCWare Provider is configured as one would configure the `qcware` client library; one must provide the URL of a Forge host and an API key.

In [ ]:
import qcware
# qcware.config.set_host(your_host_here) or set QCWARE_HOST environment variable
# qcware.config.set_api_key(your_key_here) or set QCWARE_API_KEY environment variable
def qcware_forge_probability_vector(circuit: qiskit.QuantumCircuit):
    backend = QcwareProvider().get_backend('forge_statevector')
    sv = qiskit.execute(circuit, backend).result().data()['statevector']
    return abs(sv)

In [ ]:
qiskit_circuit=circuits(min_qubits=2, max_qubits=4, min_length=2, max_length=4, gates=gates(gate_list=allowed_gates)).example()
print(qiskit_circuit.draw())
qcware_pv = qcware_forge_probability_vector(qiskit_circuit)
aer_pv = aer_probability_vector(qiskit_circuit)
atol=1e-5
print(f"PVs identical within {atol}: {numpy.allclose(qcware_pv, aer_pv, atol=atol)}")

# Measurements

Although Forge's gpu-accelerated simulator is statevector-based at heart, at very large qubit sizes (in the 20s) the added delay of transferring large amounts of data over the wire can decrease the relative benefit.  As such (and for compatibility) we provide a `forge_measurement` backend which measures the end statevector according to the gates which have been placed in the circuit.

To demonstrate this, we'll generate random circuits as before, but cap them with a `measure_all` instruction to assure results.

In [ ]:
def qcware_forge_measurements(circuit: qiskit.QuantumCircuit):
    backend = QcwareProvider().get_backend('forge_measurement')
    result = qiskit.execute(circuit, backend, shots=1000).result().data()['counts']
    return result

def aer_measurements(circuit: qiskit.QuantumCircuit):
    backend = qiskit.Aer.get_backend('qasm_simulator')
    result = qiskit.execute(circuit, backend, shots=1000).result().data()['counts']
    return result

In [ ]:
qiskit_circuit=circuits(min_qubits=2, max_qubits=4, min_length=2, max_length=4, gates=gates(gate_list=allowed_gates)).example()
qiskit_circuit.measure_all()
print(qiskit_circuit.draw())
print("Aer measurements:")
print(aer_measurements(qiskit_circuit))
print("QCWare Forge measurements:")
print(qcware_forge_measurements(qiskit_circuit))

(break here; following cells are for diagnosis of any problematic circuits if they exist)
***

In [ ]:
from qcware_transpile.translations.qiskit.to_quasar import translate
print(translate(qiskit_circuit))

In [ ]:
qiskit_circuit.data[0]